In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("Microsoft/microsoft-security-incident-prediction")
# print("Path to dataset files:", path)

100%|█████████████████████████████████████████| 513M/513M [00:28<00:00, 18.6MB/s]

Extracting files...


In [4]:
#Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy.sparse import hstack, csr_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
# import eli5
# from eli5.sklearn import PermutationImportance
import matplotlib.pyplot as plt

In [5]:
# Read aggregate file from money movement threat actors
import json

# Open the JSON file and load its contents
with open('/Users/lisasgrad-sys/Documents/SCRAM/aggregate_all.json', 'r') as file:
    json_data = json.load(file)

agg_data = dict()
for (k, v) in json_data["Techniques"].items():
    tid = v["id"]
    agg_data[tid] = {"count": v["count"], "name": k}

agg_mit = dict()
for (k, v) in json_data["Mitigations"].items():
    agg_mit[k] = {"count": v["count"]}

# Now 'data' is a Python dictionary (or list, depending on the JSON structure)
top_15 = list(agg_mit.keys())[:15]
print(top_15)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lisasgrad-sys/Documents/SCRAM/aggregate_all.json'

In [30]:
# TODO- input data as chunks
train_data = pd.read_csv('/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/GUIDE_Train.csv', nrows=10000)
test_data = pd.read_csv('/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/GUIDE_Test.csv', nrows=10000)
# Adjust pandas options to display all columns
pd.set_option('display.max_columns', None)

# Display the first few rows
train_data.head()

train_data['MitreTechniques'] = train_data['MitreTechniques'].astype('category')
test_data['MitreTechniques'] = test_data['MitreTechniques'].astype('category')

for df in [train_data, test_data]:
    # Filter for rows where IncidentGrade is 'TruePositive'
    df = df[df['IncidentGrade'] == 'TruePositive']
    # Drop rows where MitreTechniques is null
    df = df.dropna(subset=['MitreTechniques'])

    # Filter for rows where MitreTechniques contains any value from the 'agg_data' list
    filtered_df = df[df['MitreTechniques'].apply(lambda x: any(technique in x for technique in agg_data.keys()))]

    msft_counts = {}
    for tech in agg_data:
        # Create the regex pattern to match the exact technique code (e.g., T1008)
        pattern = r'\b' + tech + r'\b'
    
        # Count rows where MitreTechniques contains 'T1008'
        count = df['MitreTechniques'].str.contains(pattern).sum()
        msft_counts[tech] = {"count": count, "name": agg_data[tech]["name"]}
        
    print(msft_counts)

    # Get counts of unique values in MitreTechniques
    mitre_counts = filtered_df['MitreTechniques'].value_counts()
    # Display the counts
    print(mitre_counts)

,Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,ActionGrouped,ActionGranular,EntityType,EvidenceRole,DeviceId,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,EmailClusterId,RegistryKey,RegistryValueName,RegistryValueData,ApplicationId,ApplicationName,OAuthApplicationId,ThreatFamily,FileName,FolderPath,ResourceIdName,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City
0,180388628218,0,612,123247,2024-06-04T06:05:15.000Z,7,6,InitialAccess,NaN,TruePositive,NaN,NaN,Ip,Related,98799,138268,27,160396,441377,673934,425863,453297,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,NaN,NaN,31,6,3
1,455266534868,88,326,210035,2024-06-14T03:01:25.000Z,58,43,Exfiltration,NaN,FalsePositive,NaN,NaN,User,Impacted,98799,138268,360606,160396,22406,23032,22795,24887,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
2,1056561957389,809,58352,712507,2024-06-13T04:52:55.000Z,423,298,InitialAccess,T1189,FalsePositive,NaN,NaN,Url,Related,98799,138268,360606,68652,441377,673934,425863,453297,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
3,1279900258736,92,32992,774301,2024-06-10T16:39:36.000Z,2,2,CommandAndControl,NaN,BenignPositive,NaN,NaN,Url,Related,98799,138268,360606,13,441377,673934,425863,453297,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
4,214748368522,148,4359,188041,2024-06-15T01:08:07.000Z,9,74,Execution,NaN,TruePositive,NaN,NaN,User,Impacted,98799,138268,360606,160396,449,592,440,479,153085,529644,NaN,1631,635,860,2251,3421,881,NaN,289573,117668,3586,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630


In [78]:
train_counts = {}
test_counts = {}
for tech in agg_data:
    train_counts[tech] = {"count": 0, "name": agg_data[tech]["name"]}
    test_counts[tech] = {"count": 0, "name": agg_data[tech]["name"]}

chunk_size = 10000  # Adjust depending on your available memory

# train_data = pd.read_csv('/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/GUIDE_Train.csv', nrows=10000)
# test_data = pd.read_csv('/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/GUIDE_Test.csv', nrows=10000)

train_chunks = pd.read_csv('/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/GUIDE_Train.csv', chunksize=chunk_size)
test_chunks = pd.read_csv('/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/GUIDE_Test.csv', chunksize=chunk_size)

for (chunks, msft_counts, label) in [(train_chunks, train_counts, "TRAIN"), (test_chunks, test_counts, "TEST")]:
    for df in chunks:
        df['MitreTechniques'] = df['MitreTechniques'].astype('category')
        # Filter for rows where IncidentGrade is 'TruePositive'
        df = df[df['IncidentGrade'] == 'TruePositive']
        # Drop rows where MitreTechniques is null
        df = df.dropna(subset=['MitreTechniques'])
    
        # Filter for rows where MitreTechniques contains any value from the 'agg_data' list
        filtered_df = df[df['MitreTechniques'].apply(lambda x: any(technique in x for technique in agg_data.keys()))]

        for tech in agg_data:
            # Create the regex pattern to match the exact technique code (e.g., T1008)
            pattern = r'\b' + tech + r'\b'
        
            # Count rows where MitreTechniques contains 'T1008'
            count = df['MitreTechniques'].str.contains(pattern).sum()
            msft_counts[tech]["count"] += int(count)
    
    # sortd = OrderedDict(sorted(msft_counts.items(), key=lambda item: item[1]['count'], reverse=True))        
    # msft_counts = sortd
    # print(label, ":", msft_counts)

print(train_counts)
print(test_counts)
    # Get counts of unique values in MitreTechniques
    # mitre_counts = filtered_df['MitreTechniques'].value_counts()
    # Display the counts
    # print(mitre_counts)

{'T1588': {'count': 2, 'name': 'Obtain Capabilities'}, 'T1588.002': {'count': 0, 'name': 'Tool'}, 'T1566': {'count': 474121, 'name': 'Phishing'}, 'T1566.001': {'count': 39127, 'name': 'Spearphishing Attachment'}, 'T1204': {'count': 2902, 'name': 'User Execution'}, 'T1204.002': {'count': 1445, 'name': 'Malicious File'}, 'T1543': {'count': 2317, 'name': 'Create or Modify System Process'}, 'T1543.003': {'count': 2317, 'name': 'Windows Service'}, 'T1027': {'count': 1185, 'name': 'Obfuscated Files or Information'}, 'T1218': {'count': 790, 'name': 'System Binary Proxy Execution'}, 'T1059': {'count': 5850, 'name': 'Command and Scripting Interpreter'}, 'T1059.001': {'count': 4261, 'name': 'PowerShell'}, 'T1059.003': {'count': 951, 'name': 'Windows Command Shell'}, 'T1059.005': {'count': 612, 'name': 'Visual Basic'}, 'T1053': {'count': 316, 'name': 'Scheduled Task/Job'}, 'T1053.005': {'count': 225, 'name': 'Scheduled Task'}, 'T1562': {'count': 280, 'name': 'Impair Defenses'}, 'T1562.004': {'cou

In [83]:
import json
import scipy.stats as stats
import itertools

"""
with open("/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/train_tec_count.json", "w") as f:
    json.dump(train_counts, f)

with open("/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/test_tec_count.json", "w") as f:
    json.dump(test_counts, f)
"""
def spearmans_rank_correlation(order, label1, list1, label2, list2):
    ranked_list1 = [order.index(x) + 1 for x in list1]
    ranked_list2 = [order.index(x) + 1 for x in list2]
    # print(ranked_list1)
    # print(ranked_list2)

    # Compute Spearman's rank correlation
    correlation, p_value = stats.spearmanr(ranked_list1, ranked_list2)
    
    # Print the result
    print(f"Comparing {label1} and {label2}")
    print(f"Spearman's rank correlation: {correlation}")
    print(f"P-value: {p_value}")

# sort MSIP techniques by descending 
sortd = OrderedDict(sorted(train_counts.items(), key=lambda item: item[1]['count'], reverse=True))
train_counts = sortd

sortd = OrderedDict(sorted(test_counts.items(), key=lambda item: item[1]['count'], reverse=True))
test_counts = sortd

# comparing techniques (all)
technique_order = list(agg_data.keys())
to_compare = [("Money Movement Actors", agg_data), ("MSIP train", train_counts), ("MSIP test", test_counts)]

print("Comparing Techniques (all)")

for pair in itertools.combinations(to_compare, 2):
    ((label1, data1), (label2, data2)) = pair
    # Convert strings to ranks
    list1 = list(data1.keys())
    list2 = list(data2.keys())
    spearmans_rank_correlation(technique_order, label1, list1, label2, list2)

print("\nComparing Techniques (TOP 15)")
TOP = 15
for pair in itertools.combinations(to_compare, 2):
    ((label1, data1), (label2, data2)) = pair
    # Convert strings to ranks
    list1 = list(data1.keys())[:TOP]
    list2 = list(data2.keys())[:TOP]
    spearmans_rank_correlation(technique_order, label1, list1, label2, list2)


Comparing Techniques (all)
Comparing Money Movement Actors and MSIP train
Spearman's rank correlation: 0.4041386889684103
P-value: 2.2111537552360217e-07
Comparing Money Movement Actors and MSIP test
Spearman's rank correlation: 0.41786954016056177
P-value: 7.686738854877323e-08
Comparing MSIP train and MSIP test
Spearman's rank correlation: 0.3987307841797006
P-value: 3.309608302218478e-07

Comparing Techniques (TOP 15)
Comparing Money Movement Actors and MSIP train
Spearman's rank correlation: -0.09999999999999999
P-value: 0.7228973252791182
Comparing Money Movement Actors and MSIP test
Spearman's rank correlation: -0.035714285714285705
P-value: 0.8994469934720875
Comparing MSIP train and MSIP test
Spearman's rank correlation: 0.6357142857142856
P-value: 0.0108607461687898


In [100]:
with open("/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/train_mit_count.json", "r") as f:
    train_mit = json.load(f)

with open("/Users/lisasgrad-sys/Documents/SCRAM/microsoft-security-incident-prediction_v1/test_mit_count.json", "r") as f:
    test_mit = json.load(f)

sortd = OrderedDict(sorted(train_mit.items(), key=lambda item: item[1]['count'], reverse=True))
train_mit = sortd

sortd = OrderedDict(sorted(test_mit.items(), key=lambda item: item[1]['count'], reverse=True))
test_mit = sortd

# comparing mitigations (all)
mitigation_order = list(agg_mit.keys())

to_compare = [("Money Movement Actors", agg_mit), ("MSIP train", train_mit), ("MSIP test", test_mit)]

print("Comparing Mitigations (all)")
for pair in itertools.combinations(to_compare, 2):
    ((label1, data1), (label2, data2)) = pair
    # Convert strings to ranks
    list1 = list(data1.keys())
    list2 = list(data2.keys())
    spearmans_rank_correlation(mitigation_order, label1, list1, label2, list2)

print("\nComparing Mitigations (TOP 15)")
TOP = 15
for pair in itertools.combinations(to_compare, 2):
    ((label1, data1), (label2, data2)) = pair
    # Convert strings to ranks
    list1 = list(data1.keys())[:TOP]
    list2 = list(data2.keys())[:TOP]
    spearmans_rank_correlation(mitigation_order, label1, list1, label2, list2)

for pair in itertools.combinations(to_compare, 2):
    ((label1, data1), (label2, data2)) = pair
    list1 = list(data1.keys())[:TOP]
    list2 = list(data2.keys())[:TOP]
    print(label1, list1)
    print(label2, list2)


Comparing Mitigations (all)
Comparing Money Movement Actors and MSIP train
Spearman's rank correlation: 0.3259838786154575
P-value: 0.04895524296243798
Comparing Money Movement Actors and MSIP test
Spearman's rank correlation: 0.31460407776197247
P-value: 0.0579004463432323
Comparing MSIP train and MSIP test
Spearman's rank correlation: 0.6993835941204362
P-value: 1.4651328416253839e-06

Comparing Mitigations (TOP 15)
Comparing Money Movement Actors and MSIP train
Spearman's rank correlation: -0.28928571428571426
P-value: 0.29566536100437785
Comparing Money Movement Actors and MSIP test
Spearman's rank correlation: -0.28571428571428564
P-value: 0.3019363513225493
Comparing MSIP train and MSIP test
Spearman's rank correlation: 0.9964285714285712
P-value: 2.4159793199703526e-15
Money Movement Actors ['Execution Prevention', 'Behavior Prevention on Endpoint', 'Privileged Account Management', 'Disable or Remove Feature or Program', 'User Training', 'Restrict Web-Based Content', 'Software C

In [98]:
print(len(agg_data.keys()))
print(len(agg_mit.keys()))

153
37
